In [1]:
import os

In [2]:
%%writefile ../ENS6/dataset.py
# -*- coding: utf-8 -*-

"""
Copyright 2018 NAVER Corp.

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and
associated documentation files (the "Software"), to deal in the Software without restriction, including
without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to
the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial
portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED,
INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A
PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF
CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE
OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""

import os
import re

import numpy as np

from kor_char_parser import decompose_str_as_one_hot

class MovieReviewDataset():
    """
    영화리뷰 데이터를 읽어서, tuple (데이터, 레이블)의 형태로 리턴하는 파이썬 오브젝트 입니다.
    """
    def __init__(self, dataset_path: str, max_length: int):
        """
        initializer
        :param dataset_path: 데이터셋 root path
        :param max_length: 문자열의 최대 길이
        """
        # 데이터, 레이블 각각의 경로
        data_review = os.path.join(dataset_path, 'train', 'train_data')
        data_label = os.path.join(dataset_path, 'train', 'train_label')

        # 영화리뷰 데이터를 읽고 preprocess까지 진행합니다
        with open(data_review, 'rt', encoding='utf-8') as f:
            loaded = f.readlines()
            self.reviews_pre = preprocess_pre(loaded, max_length)
            self.reviews_post = preprocess_post(loaded, max_length)
        # 영화리뷰 레이블을 읽고 preprocess까지 진행합니다.
        with open(data_label) as f:
            self.labels = [np.float32(x) for x in f.readlines()]


def preprocess_pre(data: list, max_length: int):
    """
     입력을 받아서 딥러닝 모델이 학습 가능한 포맷으로 변경하는 함수입니다.
     기본 제공 알고리즘은 char2vec이며, 기본 모델이 MLP이기 때문에, 입력 값의 크기를 모두 고정한 벡터를 리턴합니다.
     문자열의 길이가 고정값보다 길면 긴 부분을 제거하고, 짧으면 0으로 채웁니다.

    :param data: 문자열 리스트 ([문자열1, 문자열2, ...])
    :param max_length: 문자열의 최대 길이
    :return: 벡터 리스트 ([[0, 1, 5, 6], [5, 4, 10, 200], ...]) max_length가 4일 때
    """
    def text_norm(sent):
        sent = re.sub('[\,\<\>\(\)\+\-\=\&\@\#\$]', '', sent)
        sent = re.sub('\.{2,}', '..', sent)
        sent = re.sub('\~+', '~', sent)
        sent = re.sub('\!+', '!', sent)
        sent = re.sub('\?+', '?', sent)
        sent = re.sub('ㅋ{1,}|ㅎ{1,}', 'ㅋ', sent)
        sent = re.sub('ㅜ{1,}|ㅠ{1,}|ㅠㅜ|ㅜㅠ\ㅡㅜ\ㅜㅡ\ㅡㅠ\ㅠㅡ', 'ㅠㅠ', sent)
        return sent
    
    vectorized_data = [decompose_str_as_one_hot(text_norm(datum), warning=False) for datum in data]
    zero_padding = np.zeros((len(data), max_length), dtype=np.int32)
    for idx, seq in enumerate(vectorized_data):
        length = len(seq)
        if length >= max_length:
            length = max_length
            zero_padding[idx, (max_length-length):] = np.array(seq)[:length]
        else:
            zero_padding[idx, (max_length-length):] = np.array(seq)
    return zero_padding


def preprocess_post(data: list, max_length: int):
    """
     입력을 받아서 딥러닝 모델이 학습 가능한 포맷으로 변경하는 함수입니다.
     기본 제공 알고리즘은 char2vec이며, 기본 모델이 MLP이기 때문에, 입력 값의 크기를 모두 고정한 벡터를 리턴합니다.
     문자열의 길이가 고정값보다 길면 긴 부분을 제거하고, 짧으면 0으로 채웁니다.

    :param data: 문자열 리스트 ([문자열1, 문자열2, ...])
    :param max_length: 문자열의 최대 길이
    :return: 벡터 리스트 ([[0, 1, 5, 6], [5, 4, 10, 200], ...]) max_length가 4일 때
    """
    def text_norm(sent):
        sent = re.sub('[\,\<\>\(\)\+\-\=\&\@\#\$]', '', sent)
        sent = re.sub('\.{2,}', '..', sent)
        sent = re.sub('\~+', '~', sent)
        sent = re.sub('\!+', '!', sent)
        sent = re.sub('\?+', '?', sent)
        sent = re.sub('ㅋ{1,}|ㅎ{1,}', 'ㅋ', sent)
        sent = re.sub('ㅜ{1,}|ㅠ{1,}|ㅠㅜ|ㅜㅠ\ㅡㅜ\ㅜㅡ\ㅡㅠ\ㅠㅡ', 'ㅠㅠ', sent)
        return sent
    
    vectorized_data = [decompose_str_as_one_hot(text_norm(datum), warning=False) for datum in data]
    zero_padding = np.zeros((len(data), max_length), dtype=np.int32)
    for idx, seq in enumerate(vectorized_data):
        length = len(seq)
        if length >= max_length:
            length = max_length
            zero_padding[idx, :length] = np.array(seq)[:length]
        else:
            zero_padding[idx, :length] = np.array(seq)
    return zero_padding


Overwriting ../ENS6/dataset.py


In [12]:
%%writefile ../ENS6/main.py
# -*- coding: utf-8 -*-
import argparse
import os
import numpy as np

# from sklearn.model_selection import KFold
# keras
from keras.models import Model
from keras.layers import Dense, Embedding, Input, concatenate, Flatten, add, multiply, average
from keras.layers import CuDNNLSTM, CuDNNGRU, Bidirectional, Conv1D, Dropout
from keras.layers import Dropout, SpatialDropout1D, BatchNormalization, GlobalAveragePooling1D, GlobalMaxPooling1D, PReLU
from keras.optimizers import Adam, RMSprop
from keras.layers import MaxPooling1D
from keras.layers import K, Activation
from keras.engine import Layer

##DPCNN
from keras.models import Model
from keras.layers import Input, Dense, Embedding, MaxPooling1D, Conv1D, SpatialDropout1D
from keras.layers import add, Dropout, PReLU, BatchNormalization, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import optimizers
from keras import initializers, regularizers, constraints, callbacks

from keras.callbacks import Callback
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint

import nsml
from dataset import MovieReviewDataset, preprocess_pre, preprocess_post
from nsml import DATASET_PATH, HAS_DATASET, GPU_NUM, IS_ON_NSML


# DONOTCHANGE: They are reserved for nsml
# This is for nsml leaderboard
def bind_model(model, config):
    # 학습한 모델을 저장하는 함수입니다.
    def save(filename, *args):
        model.save_weights(filename)

    # 저장한 모델을 불러올 수 있는 함수입니다.
    def load(filename, *args):
        model.load_weights(filename)
        print('Model loaded')

    def infer(raw_data, **kwargs):
        """
        :param raw_data: raw input (여기서는 문자열)을 입력받습니다
        :param kwargs:
        :return:
        """
        # dataset.py에서 작성한 preprocess 함수를 호출하여, 문자열을 벡터로 변환합니다
        preprocessed_data = preprocess_pre(raw_data, config.strmaxlen)
        # 저장한 모델에 입력값을 넣고 prediction 결과를 리턴받습니다
        point = model.predict(preprocessed_data)[-1]
        point[point>10.] = 10.
        point[point<1.] = 1.

        point = point.squeeze(axis=1).tolist()
        # DONOTCHANGE: They are reserved for nsml
        # 리턴 결과는 [(confidence interval, 포인트)] 의 형태로 보내야만 리더보드에 올릴 수 있습니다. 리더보드 결과에 confidence interval의 값은 영향을 미치지 않습니다
        return list(zip(np.zeros(len(point)), point))

    # DONOTCHANGE: They are reserved for nsml
    # nsml에서 지정한 함수에 접근할 수 있도록 하는 함수입니다.
    nsml.bind(save=save, load=load, infer=infer)

class Nsml_Callback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        nsml.report(summary=True, scope=locals(), epoch=epoch, epoch_total=config.epochs, step=epoch)
        nsml.save(epoch)

if __name__ == '__main__':
    args = argparse.ArgumentParser()
    # DONOTCHANGE: They are reserved for nsml
    args.add_argument('--mode', type=str, default='train')
    args.add_argument('--pause', type=int, default=0)
    args.add_argument('--iteration', type=str, default='0')

    # User options
    args.add_argument('--output', type=int, default=1)
    args.add_argument('--epochs', type=int, default=10)
    args.add_argument('--strmaxlen', type=int, default=150)
    
#   # args.add_argument('--maxlen', type=int, default=200)
    args.add_argument('--cell_size_l1', type=int, default=50)
    args.add_argument('--cell_size_l2', type=int, default=30)
    args.add_argument('--filter_size', type=int, default=32)
    args.add_argument('--kernel_size', type=int, default=2)
    args.add_argument('--embed_size', type=int, default=300)
    args.add_argument('--prob_dropout', type=float, default=0.5)
    args.add_argument('--max_features', type=int, default=251)
    args.add_argument('--batch_size', type=int, default=256)
    
    config = args.parse_args()

    if not HAS_DATASET and not IS_ON_NSML:  # It is not running on nsml
        DATASET_PATH = '../data/movie_review/'
        
    def get_model(config):

       

        inp = Input(shape=(config.strmaxlen, ), name='input')
        
        emb1 = Embedding(config.max_features, config.embed_size, trainable = True)(inp)
        emb2 = Embedding(config.max_features, config.embed_size, trainable = True)(inp)
        emb3 = Embedding(config.max_features, config.embed_size, trainable = True)(inp)
        
        emb1 = SpatialDropout1D(config.prob_dropout)(emb1)
        emb2 = SpatialDropout1D(config.prob_dropout)(emb2)
        emb3 = SpatialDropout1D(config.prob_dropout)(emb3)
        
        #### 
        l1_L = Bidirectional(CuDNNLSTM(config.cell_size_l1, return_sequences=True))(emb1)
        l1_G = Bidirectional(CuDNNGRU(config.cell_size_l1, return_sequences=True))(emb2)
        
        l2_LL = Bidirectional(CuDNNLSTM(config.cell_size_l2, return_sequences=True))(l1_L)
        l2_LG = Bidirectional(CuDNNGRU(config.cell_size_l2, return_sequences=True))(l1_L)
        
        l2_GL = Bidirectional(CuDNNLSTM(config.cell_size_l2, return_sequences=True))(l1_G)
        l2_GG = Bidirectional(CuDNNGRU(config.cell_size_l2, return_sequences=True))(l1_G)
        
        l3_LLC = Conv1D(config.filter_size, kernel_size = config.kernel_size, strides=1, padding = "valid", kernel_initializer = "he_uniform")(l2_LL)
        l3_LGC = Conv1D(config.filter_size, kernel_size = config.kernel_size, strides=1, padding = "valid", kernel_initializer = "he_uniform")(l2_LG)
        l3_GLC = Conv1D(config.filter_size, kernel_size = config.kernel_size, strides=1, padding = "valid", kernel_initializer = "he_uniform")(l2_GL)
        l3_GGC = Conv1D(config.filter_size, kernel_size = config.kernel_size, strides=1, padding = "valid", kernel_initializer = "he_uniform")(l2_GG)

        avg_pool_L = GlobalAveragePooling1D()(l1_L)
        max_pool_L = GlobalMaxPooling1D()(l1_L)
        
        avg_pool_G = GlobalAveragePooling1D()(l1_G)
        max_pool_G = GlobalMaxPooling1D()(l1_G)
        
        avg_pool_LL = GlobalAveragePooling1D()(l2_LL)
        max_pool_LL = GlobalMaxPooling1D()(l2_LL)
        avg_pool_LG = GlobalAveragePooling1D()(l2_LG)
        max_pool_LG = GlobalMaxPooling1D()(l2_LG)
        avg_pool_GL = GlobalAveragePooling1D()(l2_GL)
        max_pool_GL = GlobalMaxPooling1D()(l2_GL)
        avg_pool_GG = GlobalAveragePooling1D()(l2_GG)
        max_pool_GG = GlobalMaxPooling1D()(l2_GG)
        
        avg_pool_LLC = GlobalAveragePooling1D()(l3_LLC)
        max_pool_LLC = GlobalMaxPooling1D()(l3_LLC)
        avg_pool_LGC = GlobalAveragePooling1D()(l3_LGC)
        max_pool_LGC = GlobalMaxPooling1D()(l3_LGC)
        avg_pool_GLC = GlobalAveragePooling1D()(l3_GLC)
        max_pool_GLC = GlobalMaxPooling1D()(l3_GLC)
        avg_pool_GGC = GlobalAveragePooling1D()(l3_GGC)
        max_pool_GGC = GlobalMaxPooling1D()(l3_GGC)
        
        conc_LLC = concatenate([avg_pool_L, max_pool_L, avg_pool_LL, max_pool_LL, avg_pool_LLC, max_pool_LLC])
        conc_LGC = concatenate([avg_pool_L, max_pool_L, avg_pool_LG, max_pool_LG, avg_pool_LGC, max_pool_LGC])
        conc_GLC = concatenate([avg_pool_G, max_pool_G, avg_pool_GL, max_pool_GL, avg_pool_GLC, max_pool_GLC])
        conc_GGC = concatenate([avg_pool_G, max_pool_G, avg_pool_GG, max_pool_GG, avg_pool_GGC, max_pool_GGC])        

        out_LLC = Dropout(config.prob_dropout)(conc_LLC)
        out_LLC = Dense(1)(out_LLC)
        out_LGC = Dropout(config.prob_dropout)(conc_LGC)
        out_LGC = Dense(1)(out_LGC)
        out_GLC = Dropout(config.prob_dropout)(conc_GLC)
        out_GLC = Dense(1)(out_GLC)
        out_GGC = Dropout(config.prob_dropout)(conc_GGC)
        out_GGC = Dense(1)(out_GGC)
        
        ####
        c1 = Conv1D(128, kernel_size = 3, padding="valid", activation='relu', kernel_initializer = "he_uniform")(emb3)
        c2 = Conv1D(128, kernel_size = 5, padding="valid", activation='relu', kernel_initializer = "he_uniform")(emb3)
        c3 = Conv1D(128, kernel_size = 7, padding="valid", activation='relu', kernel_initializer = "he_uniform")(emb3)
        c1_avg = GlobalAveragePooling1D()(c1)
        c2_avg = GlobalAveragePooling1D()(c2)
        c3_avg = GlobalAveragePooling1D()(c3)
        c1_max = GlobalMaxPooling1D()(c1)
        c2_max = GlobalMaxPooling1D()(c2)
        c3_max = GlobalMaxPooling1D()(c3)
        
        c_conc = concatenate([c1_avg, c2_avg, c3_avg, c1_max, c2_max, c3_max])
        c_fc = BatchNormalization()(c_conc)
        c_fc = Dense(128, activation='relu')(c_fc)
        c_out = Dropout(config.prob_dropout)(c_fc)
        c_out = Dense(64, activation='relu')(c_out)
        c_out = Dropout(config.prob_dropout)(c_out)
        c_out = Dense(1)(c_out)
        
        ####
        
        #model
        #wrote out all the blocks instead of looping for simplicity
        filter_nr = 64
        filter_size = 3
        max_pool_size = 3
        max_pool_strides = 2
        dense_nr = 64
        spatial_dropout = 0.3
        dense_dropout = 0.5
        


        block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(emb3)
        block1 = BatchNormalization()(block1)
        block1 = PReLU()(block1)
        block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block1)
        block1 = BatchNormalization()(block1)
        block1 = PReLU()(block1)

        #we pass embedded comment through conv1d with filter size 1 because it needs to have the same shape as block output
        #if you choose filter_nr = embed_size (300 in this case) you don't have to do this part and can add emb_comment directly to block1_output
        resize_emb = Conv1D(filter_nr, kernel_size=1, padding='same', activation='linear')(emb3)
        resize_emb = PReLU()(resize_emb)

        block1_output = add([block1, resize_emb])
        block1_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block1_output)

        block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block1_output)
        block2 = BatchNormalization()(block2)
        block2 = PReLU()(block2)
        block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block2)
        block2 = BatchNormalization()(block2)
        block2 = PReLU()(block2)

        block2_output = add([block2, block1_output])
        block2_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block2_output)

        block3 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block2_output)
        block3 = BatchNormalization()(block3)
        block3 = PReLU()(block3)
        block3 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block3)
        block3 = BatchNormalization()(block3)
        block3 = PReLU()(block3)

        block3_output = add([block3, block2_output])
        block3_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block3_output)

        block4 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block3_output)
        block4 = BatchNormalization()(block4)
        block4 = PReLU()(block4)
        block4 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block4)
        block4 = BatchNormalization()(block4)
        block4 = PReLU()(block4)

        block4_output = add([block4, block3_output])
        block4_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block4_output)

        block5 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block4_output)
        block5 = BatchNormalization()(block5)
        block5 = PReLU()(block5)
        block5 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block5)
        block5 = BatchNormalization()(block5)
        block5 = PReLU()(block5)

        block5_output = add([block5, block4_output])
        block5_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block5_output)

        block6 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block5_output)
        block6 = BatchNormalization()(block6)
        block6 = PReLU()(block6)
        block6 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block6)
        block6 = BatchNormalization()(block6)
        block6 = PReLU()(block6)

        block6_output = add([block6, block5_output])
        block6_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block6_output)

        block7 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block6_output)
        block7 = BatchNormalization()(block7)
        block7 = PReLU()(block7)
        block7 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(block7)
        block7 = BatchNormalization()(block7)
        block7 = PReLU()(block7)

        block7_output = add([block7, block6_output])
        
        dpcnn_max = GlobalMaxPooling1D()(block7_output)
        dpcnn_avg = GlobalAveragePooling1D()(block7_output)
        
        dpcnn_conc = concatenate([dpcnn_max, dpcnn_avg])

        dpcnn_out = Dense(dense_nr, activation='linear')(dpcnn_conc)
        dpcnn_out = BatchNormalization()(dpcnn_out)
        dpcnn_out = PReLU()(dpcnn_out)
        dpcnn_out = Dropout(dense_dropout)(dpcnn_out)
        dpcnn_out = Dense(1)(dpcnn_out)
        
        ####
        
        out_avg = average([out_LLC, out_LGC, out_GLC, out_GGC, c_out, dpcnn_out])

        
# #         ==================================================================================================
        model_avg = Model(inputs=inp, outputs=[out_LLC, out_LGC, out_GLC, out_GGC, c_out, dpcnn_out, out_avg])
        
#         inp_pre = Input(shape=(config.strmaxlen, ), name='input_pre')
#         inp_post = Input(shape=(config.strmaxlen, ), name='input_post')
        
#         model_pre = model_avg(inp_pre)
#         model_post = model_avg(inp_post)
        
#         stack_layer = concatenate([model_pre, model_post])
#         ens_out = Dense(1, use_bias=False)(stack_layer)
        
#         reg_model = Model(inputs=[inp_pre, inp_post], outputs=ens_out)
        
        model_avg.compile(loss='mean_squared_error', optimizer='adam', loss_weights=[1., 1., 1., 1., 1., 1.3, 0.5] ,metrics=['mean_squared_error', 'accuracy'])
        
        return model_avg
    
    print("model creating...")
    model = get_model(config)
    model.summary()
    
    # DONOTCHANGE: Reserved for nsml use
    print("nsml binding...")
    bind_model(model, config)

    # DONOTCHANGE: They are reserved for nsml
    if config.pause:
        nsml.paused(scope=locals())

    # 학습 모드일 때 사용합니다. (기본값)
    if config.mode == 'train':
        # 데이터를 로드합니다.
        print("data loading...")
        dataset = MovieReviewDataset(DATASET_PATH, config.strmaxlen)
        
        x_pre = np.array(dataset.reviews_pre)
        x_post = np.array(dataset.reviews_post)
        y = np.array(dataset.labels)
        
        # epoch마다 학습을 수행합니다.
        nsml_callback = Nsml_Callback()
#         dataset_val = MovieReviewDataset_val(DATASET_PATH, config.strmaxlen)
#         x_val = np.array(dataset_val.reviews)
#         y_val = np.array(dataset_val.labels)
        print("model training...")
        hist = model.fit(x_pre, [y,y,y,y,y,y,y], 
                         validation_split = 0.1,
                         batch_size=config.batch_size, callbacks=[nsml_callback], epochs=config.epochs, verbose=2)


    # 로컬 테스트 모드일때 사용합니다
    # 결과가 아래와 같이 나온다면, nsml submit을 통해서 제출할 수 있습니다.
    # [(0.0, 9.045), (0.0, 5.91), ... ]
    elif config.mode == 'test_local':
        with open(os.path.join(DATASET_PATH, 'test/test_data'), 'rt', encoding='utf-8') as f:
            reviews = f.readlines()
        res = nsml.infer(reviews)
        print(res)

Overwriting ../ENS6/main.py


 ===